In [ ]:
# === Load Payloads ===
def load_payloads_from_folder(folder):
    """
    Load text payloads from a specified folder.

    Args:
        folder (str): Path to the folder containing payload files.

    Returns:
        list: A list of payload contents as strings.

    Notes:
        - Files are read in sorted order.
        - Uses 'errors=ignore' to handle encoding issues.
    """
    payloads = []
    for file in sorted(os.listdir(folder)):
        path = os.path.join(folder, file)
        with open(path, 'r', errors='ignore') as f:
            content = f.read()
            payloads.append(content)
    return payloads


In [ ]:

# === LSB Steganography Function ===
def generate_lsb_stego(image_path, payload, output_path, bpp=0.2):
    """
    Embed a payload into a cover image using Least Significant Bit (LSB) steganography.

    Args:
        image_path (str): Path to the input cover image.
        payload (str): Text payload to embed into the image.
        output_path (str): Path to save the resulting stego image.
        bpp (float, optional): Bits per pixel ratio for embedding (default: 0.2).

    Notes:
        - Converts the image to grayscale ('L' mode).
        - Limits payload size based on bpp and image capacity.
        - Embeds binary payload into the least significant bit of each pixel.
        - Saves the stego image as a PNG.
    """
    # Load and convert the image to grayscale
    img = Image.open(image_path).convert('L')
    arr = np.array(img)

    # Flatten the image array for easier manipulation
    flat = arr.flatten()
    max_payload_bits = int(len(flat) * bpp)

    # Convert payload to binary string
    binary_payload = ''.join(f'{ord(c):08b}' for c in payload)
    binary_payload = binary_payload[:max_payload_bits]

    # Pad with zeros if payload is shorter than max capacity
    if len(binary_payload) < max_payload_bits:
        binary_payload += '0' * (max_payload_bits - len(binary_payload))

    # Embed the payload into the least significant bit of each pixel
    for i in range(max_payload_bits):
        flat[i] = (flat[i] & ~1) | int(binary_payload[i])

    # Reshape the modified array back to the original image dimensions
    stego_arr = flat.reshape(arr.shape)


    stego_img = Image.fromarray(stego_arr.astype(np.uint8))
    stego_img.save(output_path)


In [ ]:
# === Main Execution ===
from google.colab import drive
import os
from PIL import Image
import numpy as np

drive.mount('/content/drive', force_remount=True)

base_path = '/content/drive/MyDrive/Stego-Images-Dataset/'
cover_path = os.path.join(base_path, 'cover')  # Directory containing cover images
output_path = os.path.join(base_path, 'stego/LSB/')  # Directory to save stego images
os.makedirs(output_path, exist_ok=True)

# folders containing malicious payloads
ps_folder = os.path.join(base_path, 'Malicious_PS_Samples')  # PowerShell scripts
js_folder = os.path.join(base_path, 'Malicious_JS_Samples')  # JavaScript files
url_folder = os.path.join(base_path, 'Malicious_URL_Samples')  # URLs

# Load payloads from each folder
ps_payloads = load_payloads_from_folder(ps_folder)
js_payloads = load_payloads_from_folder(js_folder)
url_payloads = load_payloads_from_folder(url_folder)

# Combine all payloads into a single list
payloads = ps_payloads + js_payloads + url_payloads

# Embed payloads into cover images
cover_files = sorted(os.listdir(cover_path))  # List cover images in sorted order
for i, file in enumerate(cover_files):
    if i >= len(payloads):  # Stop if there are no more payloads to embed
        break
    input_img_path = os.path.join(cover_path, file)  # Path to the current cover image
    output_img_path = os.path.join(output_path, file)  # Path to save the stego image
    # Generate and save the stego image with the corresponding payload
    generate_lsb_stego(input_img_path, payloads[i], output_img_path, bpp=0.2)